In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, RepeatVector
from tensorflow.keras import Input, Model

# Set the path to the segmented data folder
segmented_data_folder = 'segmented_data'

# Initialize lists to hold data and labels
data = []
labels = []

# Load the segmented CSV files
for file_name in os.listdir(segmented_data_folder):
    file_path = os.path.join(segmented_data_folder, file_name)
    
    # Read the CSV file
    segment = pd.read_csv(file_path)
    
    # Append the segment data
    data.append(segment.values)
    
    # Determine the label from the filename ('H' for hypertensive, 'N' for normal)
    label = 1 if 'H' in file_name else 0
    labels.append(label)

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


In [2]:
# Build an autoencoder model
def build_autoencoder(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    #model.add(LSTM(128, activation='relu', return_sequences=True))
    model.add(LSTM(64, activation='relu', return_sequences=False))
    model.add(RepeatVector(input_shape[0]))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    #model.add(LSTM(128, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(input_shape[1])))
    model.compile(optimizer='adam', loss='mse')
    return model

# Train the autoencoder
input_shape = (X_train.shape[1], X_train.shape[2])  # Segment length and number of features
autoencoder = build_autoencoder(input_shape)



In [ ]:
autoencoder.fit(X_train, X_train, epochs=10, batch_size=64, validation_data=(X_test, X_test))

# Extract features using the encoder part of the autoencoder
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[2].output)
encoded_X_train = encoder.predict(X_train)
encoded_X_test = encoder.predict(X_test)

Epoch 1/10


In [ ]:
# Build an LSTM model for classification
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(encoded_X_train.shape[1], encoded_X_train.shape[2])))
lstm_model.add(Dense(1, activation='sigmoid'))
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the LSTM model
lstm_model.fit(encoded_X_train, y_train, epochs=50, batch_size=32, validation_data=(encoded_X_test, y_test))

# Evaluate the model
loss, accuracy = lstm_model.evaluate(encoded_X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
